In [162]:
import pandas as pd
import numpy as np
import torch
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
from DataSet import DataSet
from models.VanillaFFNN import FFNN
from datetime import datetime

In [163]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [164]:
seed = 2024
np.random.seed(seed)

In [165]:
df = pd.read_csv('data/computed_values.csv')
solution_columns = [col for col in df.columns if col.startswith('Solution_')]
num_bids = len(solution_columns)
num_goods = (len(df.columns) - num_bids) // num_bids - 1
df.head()

,Price_0,Bid_0_0,Bid_0_1,Bid_0_2,Bid_0_3,Bid_0_4,Bid_0_5,Bid_0_6,Bid_0_7,Bid_0_8,...,Solution_4,Solution_5,Solution_6,Solution_7,Solution_8,Solution_9,Solution_10,Solution_11,Solution_12,Solution_13
0,0.097675,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.117325,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.086037,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.100991,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.116555,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0


In [166]:
len(df)

1000

In [167]:
hparams = {
    'batch_size': 1,
    'learning_rate': 1e-3,
    'epochs': 60,
    'num_workers': 0,
    'input_size': num_bids * (num_goods + 1),
    'hidden_size': 256,
    'output_size': num_bids
}

In [168]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [169]:
data = DataSet(df, num_bids)
train_data = torch.utils.data.Subset(data, range(0, 800))
val_data = torch.utils.data.Subset(data, range(800, 900))
test_data = torch.utils.data.Subset(data, range(900, 1000))
train_loader = DataLoader(train_data, batch_size=hparams['batch_size'], shuffle=True,
                          num_workers=hparams['num_workers'])
val_loader = DataLoader(val_data, batch_size=hparams['batch_size'], shuffle=True, num_workers=hparams['num_workers'])
test_loader = DataLoader(test_data, batch_size=hparams['batch_size'], shuffle=True, num_workers=hparams['num_workers'])
# train_data[0]

In [170]:
# Test data loader
for i, (x, y) in enumerate(train_loader):
    print(x.shape)
    print(y.shape)
    print(x)
    print(y)
    break

torch.Size([1, 168])
torch.Size([1, 14])
tensor([[0.1553, 1.0000, 1.0000, 1.0000, 1.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.1178, 0.0000, 1.0000, 0.0000, 0.0000, 1.0000,
         1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0767, 1.0000, 1.0000,
         0.0000, 0.0000, 0.0000, 1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.2275, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0878, 0.0000, 0.0000, 1.0000, 1.0000, 1.0000,
         0.0000, 1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.1038, 0.0000, 1.0000,
         1.0000, 0.0000, 1.0000, 0.0000, 1.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0848, 1.0000, 0.0000, 1.0000, 1.0000, 0.0000, 0.0000, 1.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0608, 1.0000, 0.0000, 0.0000, 1.0000, 1.0000,
         0.0000, 1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0543, 0.0000, 0.0000,
         0.0000, 0.0000, 1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0

/home/kassian/Alles/Uni/Master/Semester-2/IDP/machine-learning-for-auctions/DataSet.py:21: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  self.data.iloc[idx].iloc[-self.num_bids:])


In [171]:
# Create model, loss function and optimizer
writer = SummaryWriter('runs/FFNN' + str(datetime.now()))
model = FFNN(hparams['input_size'], hparams['hidden_size'], hparams['output_size'])
model.to(device)
loss_fn = torch.nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=hparams['learning_rate'])

Parameter containing:
tensor([[ 0.0339,  0.0206, -0.0123,  ...,  0.0239, -0.0520, -0.0239],
        [-0.0115,  0.0737, -0.0464,  ...,  0.0536,  0.0048, -0.0394],
        [-0.1049,  0.0931,  0.0092,  ...,  0.0311,  0.1348,  0.0484],
        ...,
        [ 0.1364,  0.0225,  0.0040,  ..., -0.0700, -0.0603, -0.1166],
        [-0.0422, -0.0424,  0.1195,  ...,  0.0550,  0.0474,  0.0949],
        [ 0.0249,  0.0455, -0.0782,  ..., -0.0962, -0.0199, -0.1476]],
       requires_grad=True)


In [172]:
def train_model(model, loss_fn, optimizer, train_loader, val_loader, patience=5, epochs=20):
    best_loss = np.inf
    patience_counter = 0

    for epoch in range(epochs):
        model.train()
        train_loss = 0
        # print(str(epoch) + " " + str(model.fc1.weight))
        for i, (x, y) in enumerate(train_loader):
            x = x.float().to(device)
            y = y.float().to(device)
            y_pred = model(x)
            loss = loss_fn(y_pred, y)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            train_loss += loss.item()

        train_loss /= len(train_loader)
        writer.add_scalar('Loss/train', train_loss, epoch)

        model.eval()
        val_loss = 0
        with torch.no_grad():
            for i, (x, y) in enumerate(val_loader):
                x = x.float().to(device)
                y = y.float().to(device)
                y_pred = model(x)
                loss = loss_fn(y_pred, y)
                val_loss += loss.item()

        val_loss /= len(val_loader)
        writer.add_scalar('Loss/val', val_loss, epoch)

        print(f'Epoch: {epoch}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}')

        if val_loss < best_loss:
            best_loss = val_loss
            torch.save(model.state_dict(), 'best_model.pth')
            patience_counter = 0
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print("Early stopping")
                break


hparams = {
    'batch_size': 8,
    'learning_rate': 1e-3,
    'epochs': 20,
    'num_workers': 0,
    'input_size': num_bids * (num_goods + 1),
    'hidden_size': 256,
    'output_size': num_bids,
    'patience': 5
}
train_model(model, loss_fn, optimizer, train_loader, val_loader, epochs=hparams['epochs'], patience=hparams['patience'])

Epoch: 0, Train Loss: 0.2826, Val Loss: 0.2516
Epoch: 1, Train Loss: 0.2543, Val Loss: 0.2376
Epoch: 2, Train Loss: 0.2404, Val Loss: 0.2367
Epoch: 3, Train Loss: 0.2304, Val Loss: 0.2538
Epoch: 4, Train Loss: 0.2211, Val Loss: 0.2345
Epoch: 5, Train Loss: 0.2155, Val Loss: 0.2428
Epoch: 6, Train Loss: 0.2111, Val Loss: 0.2371
Epoch: 7, Train Loss: 0.2082, Val Loss: 0.2437
Epoch: 8, Train Loss: 0.2046, Val Loss: 0.2431
Epoch: 9, Train Loss: 0.2004, Val Loss: 0.2442
Early stopping


In [173]:
def visualize(bits, prediction):
    solution = bits[1]
    bits = bits[0]
    print(prediction)
    print(solution)
    for i in range(num_bids):
        if bits[i * (num_goods + 1)] == 0:
            break
        print(f'Bid {i}: Price {bits[i * (num_goods + 1)]}')
        good_list = []
        for j in range(num_goods):
            if bits[i * (num_goods + 1) + j + 1] == 1:
                good_list.append(j)
        print(f'       Goods {good_list}')
        if solution[i] == 1:
            if prediction[i] == 1:
                print(f'       BOTH')
            else:
                print(f'       Only solution')
        elif prediction[i] == 1:
            print(f'       Only prediction')

    print("Equal: " + str(torch.all(torch.eq(prediction, solution))))

In [174]:
# Test model
test_object = test_data[3]

model = FFNN(hparams['input_size'], hparams['hidden_size'], hparams['output_size'])
model.load_state_dict(torch.load('best_model.pth'))
model.eval()  # Set the model to evaluation mode

visualize(test_object, model(test_object[0].float().to(device)).round())


Parameter containing:
tensor([[ 0.0188,  0.0394, -0.0788,  ..., -0.0186, -0.0517, -0.0897],
        [ 0.0707, -0.0296, -0.0990,  ...,  0.0398, -0.0018, -0.0550],
        [-0.0008, -0.1153, -0.0080,  ...,  0.0756, -0.1039, -0.0461],
        ...,
        [ 0.0663,  0.0073,  0.0143,  ...,  0.0235,  0.0247,  0.0516],
        [-0.0871, -0.0754,  0.0274,  ..., -0.1761,  0.0131,  0.1092],
        [ 0.0586,  0.0436,  0.0151,  ...,  0.0344, -0.0055,  0.0374]],
       requires_grad=True)
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       grad_fn=<RoundBackward0>)
tensor([0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       dtype=torch.float64)
Bid 0: Price 0.0791307496363301
       Goods [1, 3, 5]
Bid 1: Price 0.073410781490287
       Goods [0, 1, 5]
Bid 2: Price 0.0784781939568724
       Goods [1, 4, 6]
Bid 3: Price 0.0862580436149736
       Goods [3, 4, 6]
Bid 4: Price 0.0717886320180775
       Goods [0, 1, 6]
Bid 5: Price 0.2195341676421315
       Goods [0, 1, 

In [175]:
model.eval()
wrong = 0
correct = 0
for i, (x, y) in enumerate(train_loader):
    x = x.float().to(device)
    y = y.float().to(device)
    y_pred = model(x)
    if not torch.all(torch.eq(y_pred.round(), y)):
        wrong += 1
    else:
        correct += 1

print("Correct: " + str(correct))
print("Wrong: " + str(wrong))

Correct: 345
Wrong: 455
